Model of Breast Cancer Prediction was created based on the kaggle dataset and notebook: https://www.kaggle.com/smohemmedsuhail/detail-breast-cancer-prediction/notebook. This model will be used for the pet-project deployment

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

from sklearn.feature_selection import RFE

from joblib import dump
from joblib import load

In [2]:
df = pd.read_csv('cancer.csv')
df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


In [3]:
# dropping fileds that we do not need
df.drop(['id'],axis=1, inplace=True)
df.drop(['Unnamed: 32'],axis=1, inplace=True)

df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    object 
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

In [5]:
# Encoding of target field
le = LabelEncoder()
df['diagnosis'] = le.fit_transform(df.diagnosis)

In [6]:
# plit dataset
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('diagnosis', axis=1), df.diagnosis, test_size=0.20, random_state=42)

In [7]:
#  there are 30 fileds in the dataset. We will use only 13 of them. The best columns remain with using of RFE
rfe = RFE(LogisticRegression(), n_features_to_select=len(X_train.columns)-17)   
rfe = rfe.fit(X_train, y_train)

X_train_rfe = X_train[X_train.columns[rfe.support_]]
X_test_rfe = X_test[X_test.columns[rfe.support_]]

C:\Users\fadee\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\fadee\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\fadee\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
X_test_rfe.head()

,radius_mean,texture_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,texture_se,perimeter_se,radius_worst,texture_worst,concavity_worst,concave points_worst,symmetry_worst
204,12.47,18.60,0.1058,0.08005,0.03821,0.1925,1.0440,2.497,14.97,24.64,0.2671,0.10150,0.3014
70,18.94,21.31,0.1029,0.10800,0.07951,0.1582,0.7975,5.486,24.86,26.58,0.2687,0.17890,0.2551
131,15.46,19.48,0.1223,0.14660,0.08087,0.1931,0.7859,3.094,19.26,26.00,0.3791,0.15140,0.2837
431,12.40,17.68,0.1316,0.07741,0.02799,0.1811,1.4600,2.204,12.88,22.91,0.2403,0.07370,0.2556
540,11.54,14.44,0.1120,0.06737,0.02594,0.1818,1.7680,1.628,12.26,19.68,0.1797,0.06918,0.2329


In [9]:
#  Lets check which model can be the best based on accuracy
models = [LogisticRegression(), RandomForestClassifier(),
          GradientBoostingClassifier()]
for model in models:
    print(str(model), np.mean(cross_val_score(model,
          X_train_rfe, y_train, cv=3, scoring='accuracy')))

C:\Users\fadee\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\fadee\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

LogisticRegression() 0.9515946322760543
RandomForestClassifier() 0.9560241663762055
GradientBoostingClassifier() 0.9604246543511096


In [10]:
# Lets find the best parameters for the model
params = {
    'n_estimators':[50,70,100,150,200],
    'max_depth':[3,6,9]
    
}
rsCV = RandomizedSearchCV(RandomForestClassifier(),params)
rsCV.fit(X_train_rfe, y_train)


RandomizedSearchCV(estimator=RandomForestClassifier(),
                   param_distributions={'max_depth': [3, 6, 9],
                                        'n_estimators': [50, 70, 100, 150,
                                                         200]})

In [11]:
rsCV.best_params_

{'n_estimators': 150, 'max_depth': 6}

In [12]:
predict = rsCV.best_estimator_.predict(X_test_rfe)
accuracy_score(y_test, predict)

0.956140350877193

## Model saving

In [13]:
dump(rsCV.best_estimator_, 'model_c.joblib') 

['model_c.joblib']

In [14]:
np.save('classes_c.npy', le.classes_)

In [17]:
X_test_rfe.columns

Index(['radius_mean', 'texture_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'texture_se', 'perimeter_se',
       'radius_worst', 'texture_worst', 'concavity_worst',
       'concave points_worst', 'symmetry_worst'],
      dtype='object')

## Model usage

In [18]:
# function to get prediction from based on the inputs
def get_predictions(rm, tm, cm, cnm, cpm, sm, ts, ps, rw, tw, cw, cpw, sw):
    cols = ['radius_mean', 'texture_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'texture_se', 'perimeter_se',
       'radius_worst', 'texture_worst', 'concavity_worst',
       'concave points_worst', 'symmetry_worst']

    model = load('model_c.joblib')

    encoder = LabelEncoder()
    encoder.classes_ = np.load('classes_c.npy', allow_pickle=True)

    df = pd.DataFrame([[rm, tm, cm, cnm, cpm, sm, ts, ps, rw, tw, cw, cpw, sw]], columns=cols)

    if model.predict(df)[0] == 0:
        return 'Benign'
    else:
        return 'Malignant'

In [19]:
get_predictions(11.54, 14.44, 76.80, 0.07871, 0.02904, 0.2784, 1.7680, 12.38, 19.33, 0.1622, 0.2118, 0.1797, 0.11890)

'Malignant'